### Import

In [43]:
import os
import json
import requests
from bs4 import BeautifulSoup
from deep_translator import GoogleTranslator

1. Utils

In [44]:
def extract(ancestor, selector, attribute = None, multiple = False):
    if selector:
        if multiple:
            if attribute:
                return [tag[attribute] for tag in ancestor.select(selector)]
            return [tag.get_text().strip("\n") for tag in ancestor.select(selector)] 
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute]
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).get_text()
        except AttributeError:
            return None
    try:
        return ancestor[attribute]
    except (KeyError, KeyError):
        return None
    

In [45]:
def translate(text, source = "pl", target = "en"):
    return GoogleTranslator(source, target).translate(text = text)

In [46]:
selectors = {
    "opinion_id": ( None, "data-entry-id"),
    "author" : ("span.user-post__author-name",),
    "recommendation": ("span.user-post__author-recomendation > em",),
    "recommendation" : (None,),
    "stars" :("span.user-post__score-count",),
    "content_pl" :( "div.user-post__text",),
    "pros_pl" : ("div.review-feature__item--positive", None, True),
    "cons_pl" : ("div.review-feature__item--negative", None, True),
    "vote_yes" :( "button.vote-yes","data-total-vote"),
    "vote_no" : ("button.vote-no","data-total-vote"),
    "published" :( "span.user-post__published > time:nth-child(1)","datetime"),
    "purchased" : ("span.user-post__published > time:nth-child(2)","datetime")
}

2. Extration of opinions

In [47]:
with open("./cookie.json", "r") as jf:
    headers = json.load(jf)

3. If there are more pages, move to the next page and repert 2 - 6 steps

In [48]:



product_id = input("Enter produc code")
next_page = f"https://www.ceneo.pl/{product_id}#tab-reviews_scroll"

all_opinions = []

while next_page:
    response = requests.get(next_page, headers = headers)
    if response.status_code == 200:
        page_dom = BeautifulSoup(response.text, 'html.parser')
        opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")
        for opinion in opinions:
            single_opinion = {
                key: extract(opinion, *value)
                for key, value in selectors.items()
            }
            single_opinion['content_en'] = translate(single_opinion['content_pl'])
            single_opinion['pros_en'] = [translate(pros) for pros in single_opinion['pros_pl']]
            single_opinion['cons_en'] = [translate(cons) for cons in single_opinion['cons_pl']]
            single_opinion['recommendation'] = True if single_opinion['recommendation'] == "Polecam" else False if single_opinion['recommendation'] == " Nie polecam" else None
            single_opinion['stars'] = float(single_opinion['stars'].split("/")[0].replace(",","."))
            single_opinion['vote_yes'] = int(single_opinion['vote_yes'])
            single_opinion['vote_yes'] = int(single_opinion['vote_no'])
            all_opinions.append(single_opinion)
            
        try:
            next_page = "https://www.ceneo.pl" + page_dom.select_one("a.pagination__next")["href"]
        except TypeError:
            next_page = None
    else:
        print("NOT OK")


In [49]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")
with open(f"./opinions/{product_id}.json","w", encoding = "UTF-8") as jf:
    json.dump(all_opinions, jf, ensure_ascii = False, indent = 4)